In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [27]:
data=pd.read_csv('data.csv')
data.head()

,client_email,freelancer_email,service,project_name,review,sentiment,rating
0,nishtha@gmail.com,kaushik@gmail.com,ppt,Project 0,The ppt developed by kaushik@gmail.com had sev...,Positive,5
1,test@gmail.com,anushka@gmail.com,figma,Project 1,I appreciate the effort put into the figma pro...,Neutral,3
2,anushka@gmail.com,nishtha@gmail.com,website,Project 2,I'm indifferent about the website project prov...,Negative,1
3,kaushik@gmail.com,nishtha@gmail.com,backend,Project 3,The backend work done by nishtha@gmail.com sur...,Neutral,3
4,nishtha@gmail.com,test@gmail.com,backend,Project 4,The backend provided by test@gmail.com was sat...,Neutral,3


In [28]:
newData=data.sort_values('rating')
newData.head()
newData.tail()

,client_email,freelancer_email,service,project_name,review,sentiment,rating
483,raghav@gmail.com,raghav@gmail.com,backend,Project 483,I'm extremely satisfied with the backend devel...,Positive,5
498,nishtha@gmail.com,anushka@gmail.com,figma,Project 498,The figma developed by anushka@gmail.com excee...,Positive,5
506,kaushik@gmail.com,test@gmail.com,backend,Project 506,I'm very satisfied with the backend created by...,Positive,5
419,nishtha@gmail.com,test@gmail.com,app,Project 419,The app developed by test@gmail.com exceeded m...,Positive,5
0,nishtha@gmail.com,kaushik@gmail.com,ppt,Project 0,The ppt developed by kaushik@gmail.com had sev...,Positive,5


In [29]:
newData['rating'].value_counts()

3    704
5     91
1     73
4     72
2     60
Name: rating, dtype: int64

In [30]:
newData['rating']
# print(newData['review'][0])

423    1
490    1
904    1
460    1
907    1
      ..
483    5
498    5
506    5
419    5
0      5
Name: rating, Length: 1000, dtype: int64

In [31]:
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentiment=SentimentIntensityAnalyzer()
new_data=pd.DataFrame()
for i in range(0,1000):
    # print(sentences)
    sentences=data['review'][i]
    rating=data['rating'][i]
    res=sentiment.polarity_scores(sentences)
    score=res['compound']
    if score>0.5:
        score=score*res['pos']
    elif score<-0.5:
        score=score*res['neg']
    else:
        score=score*res['neu']

    if score>0.25:
        if rating > 3:
            new_data=new_data.append({'review':sentences,'rating':rating,'sentiment':'Positive'},ignore_index=True)
    elif score<-0.25:
        if rating < 3 :
            new_data=new_data.append({'review':sentences,'rating':rating,'sentiment':'Negative'},ignore_index=True) 
    else:
        if rating==3:  
            new_data=new_data.append({'review':sentences,'rating':rating,'sentiment':'Neutral'},ignore_index=True)

new_data.head()

#Save new data to csv
new_data.to_csv('new_data.csv',index=False)



C:\Users\Raghav\AppData\Local\Temp\ipykernel_21904\1922096640.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_data=new_data.append({'review':sentences,'rating':rating,'sentiment':'Neutral'},ignore_index=True)
C:\Users\Raghav\AppData\Local\Temp\ipykernel_21904\1922096640.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_data=new_data.append({'review':sentences,'rating':rating,'sentiment':'Neutral'},ignore_index=True)
C:\Users\Raghav\AppData\Local\Temp\ipykernel_21904\1922096640.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_data=new_data.append({'review':sentences,'rating':rating,'sentiment':'Neutral'},ignore_index=True)
C:\Users\Raghav\AppData\Local\Temp\ipykernel_21904\1922096640.py:23: FutureWarn

In [32]:
new_data.value_counts('rating')

rating
3    422
4     37
5     35
1     21
2     17
dtype: int64

In [33]:
new_data.shape

(532, 3)

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier


In [35]:
X=new_data['review']
y=new_data['sentiment'].replace({'Positive':1,'Negative':0,'Neutral':-1})
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

tfidf=TfidfVectorizer()
X_train=tfidf.fit_transform(X_train)
X_test=tfidf.transform(X_test)

logisticModel=LogisticRegression()
linearModel=LinearRegression()
randomModel=RandomForestClassifier()
naiveModel=MultinomialNB()
decisionModel=DecisionTreeClassifier()
knnModel=KNeighborsClassifier()
adaModel=AdaBoostClassifier()
gradientModel=GradientBoostingClassifier()
baggingModel=BaggingClassifier()
extraModel=ExtraTreesClassifier()



In [36]:
models=[logisticModel,randomModel,naiveModel,decisionModel,knnModel,adaModel,gradientModel,baggingModel,extraModel]
print(X_train.shape)
for model in models:
    model.fit(X_train,y_train)
    y_pred=model.predict(X_test)
    print(f'Accuracy score of {model} is \n {accuracy_score(y_test,y_pred)}')
    print('------------------------------------------------------')


votingModel=VotingClassifier(estimators=[('logistic',logisticModel),('random',randomModel),('naive',naiveModel),('decision',decisionModel),('knn',knnModel),('ada',adaModel),('gradient',gradientModel),('bagging',baggingModel),('extra',extraModel)],voting='hard')
votingModel.fit(X_train,y_train)
y_pred=votingModel.predict(X_test)
print(f'Accuracy score of {votingModel} is {accuracy_score(y_test,y_pred)}')


(425, 95)
Accuracy score of LogisticRegression() is 
 1.0
------------------------------------------------------
Accuracy score of RandomForestClassifier() is 
 1.0
------------------------------------------------------
Accuracy score of MultinomialNB() is 
 1.0
------------------------------------------------------
Accuracy score of DecisionTreeClassifier() is 
 1.0
------------------------------------------------------
Accuracy score of KNeighborsClassifier() is 
 1.0
------------------------------------------------------
Accuracy score of AdaBoostClassifier() is 
 1.0
------------------------------------------------------
Accuracy score of GradientBoostingClassifier() is 
 1.0
------------------------------------------------------
Accuracy score of BaggingClassifier() is 
 1.0
------------------------------------------------------
Accuracy score of ExtraTreesClassifier() is 
 1.0
------------------------------------------------------
Accuracy score of VotingClassifier(estimators=[('

In [37]:
new_data['sentiment'].value_counts()

Neutral     422
Positive     72
Negative     38
Name: sentiment, dtype: int64

In [38]:
# import joblib
# joblib.dump(votingModel,'reviews.joblib')

In [39]:
#Input the review
review=input('Enter the review: ')
review=[review]
review=tfidf.transform(review)
sentiment=votingModel.predict(review)
if sentiment==1:
    print('Positive review')
elif sentiment==0:
    print('Negative review')
else:
    print('Neutral review')

Neutral review


In [44]:
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
text=TextBlob('The ppt work done by anushka@gmail.com was subpar. I regret hiring them for this project')
print(text.sentiment)

Sentiment(polarity=0.0, subjectivity=0.0)
